In [91]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings(action='ignore')
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 7.5)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [69]:
df_engagement = pd.read_csv('takehome_user_engagement.csv')
df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [8]:
df_users = pd.read_csv('takehome_users.csv', encoding='latin-1')
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [70]:
df_engagement['time_stamp'] = pd.to_datetime(df_engagement['time_stamp'])
df_engagement.drop(['visited'], axis=1, inplace=True)

In [71]:
grouped_users = pd.DataFrame(df_engagement.groupby('user_id')['time_stamp'].apply(list))
grouped_index = list(grouped_users.index)
grouped_timestamps = list(grouped_users['time_stamp'])

In [72]:
def is_adopted(timestamps):
    '''Takes in a list of datetimes and returns 1 if the user is adopted, 
    and 0 if the user is not adopted'''
    
    timestamps = pd.DataFrame(timestamps)
    timestamps[0] = pd.to_datetime(timestamps[0])
    timecounts = timestamps.set_index(0).sort_index().resample('1W').size()
    
    for i in timecounts:
        if i >= 3:
            return 1
    return 0

In [73]:
# weekly = df_engagement.set_index('time_stamp').sort_index().resample('1W').size()
adapted = list(map(is_adopted, grouped_timestamps))

In [75]:
df_adopted = pd.DataFrame({'object_id':grouped_index, 'adopted':adapted})
df_adopted.head()

,object_id,adopted
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [76]:
df_clean = pd.merge(df_users, df_adopted, on='object_id', how='outer')
df_clean.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0


In [77]:
df_clean.drop(['email'], axis=1, inplace=True)
df_clean.head()

,object_id,creation_time,name,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
0,1,2014-04-22 03:53:30,Clausen August,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0


In [78]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
adopted                       8823 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 1.0+ MB


In [95]:
df_clean['last_session_creation_time'] = df_clean['last_session_creation_time'].map(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
df_clean.head()

TypeError: an integer is required (got type Timestamp)